# Fusão de Dataframes


#### 1 - Carregar Dataframe

In [ ]:
import numpy as np
import pandas as pd

df = pd.DataFrame([{'Name': 'Chris', 'Item Purchased': 'Sponge', 'Cost': 22.50},
                   {'Name': 'Kevyn', 'Item Purchased': 'Kitty Litter', 'Cost': 2.50},
                   {'Name': 'Filip', 'Item Purchased': 'Spoon', 'Cost': 5.00}],
                  index=['Store 1', 'Store 1', 'Store 2'])
df

#### 2 - Adicionar uma nova coluna (mesmo comprimento)

In [ ]:
df['Date'] = ['December 1', 'January 1', 'mid-day']
df

#### 3 - Flag de entraga do produto

In [ ]:
df['Deliverd'] = True
df

#### 4 - Fornecer uma lista tão longa quanto o Dataframe

In [ ]:
df['Feedback'] = ['Positive', None, 'Negative']
df

#### 5 - Reiniciando índices

In [ ]:
adf = df.reset_index()
adf['Date'] = pd.Series({0: 'December 1', 2: 'mid-May'})
adf

#### 6 - Criação de dois Dataframes com sobreposição

In [ ]:
staff_df = pd.DataFrame([{'Name': 'Kelly', 'Role': 'Director of HR'},
                         {'Name': 'Sally', 'Role': 'Course liasion'},
                         {'Name': 'James', 'Role': 'Grader'}])
staff_df = staff_df.set_index('Name')
student_df = pd.DataFrame([{'Name': 'James', 'School': 'Business'},
                           {'Name': 'Mike', 'School': 'Law'},
                           {'Name': 'Sally', 'School': 'Engineering'}])
student_df = student_df.set_index('Name')
print(staff_df.head())
print()
print(student_df.head())

#### 7 - Outer Join

In [ ]:
pd.merge(staff_df, student_df, how='outer', left_index=True, right_index=True)

#### 8 - Inner Join

In [ ]:
pd.merge(staff_df, student_df, how='inner', left_index=True, right_index=True)

#### 9 - Left Join

In [ ]:
pd.merge(staff_df, student_df, how='left', left_index=True, right_index=True)

#### 10 - Right Join

In [ ]:
pd.merge(staff_df, student_df, how='right', left_index=True, right_index=True)

#### 11 - Fusão por colunas

In [ ]:
staff_df = staff_df.reset_index()
student_df = student_df.reset_index()
pd.merge(staff_df, student_df, how='left', left_on='Name', right_on='Name')

#### 12 - Conflitos

In [ ]:
staff_df = pd.DataFrame([{'Name': 'Kelly', 'Role': 'Director of HR', 'Location': 'State Street'},
                         {'Name': 'Sally', 'Role': 'Course liasion', 'Location': 'Washington Avenue'},
                         {'Name': 'James', 'Role': 'Grader', 'Location': 'Washington Avenue'}])
student_df = pd.DataFrame([{'Name': 'James', 'School': 'Business', 'Location': '1024 Billiard Avenue'},
                           {'Name': 'Mike', 'School': 'Law', 'Location': 'Fraternity House #22'},
                           {'Name': 'Sally', 'School': 'Engineering', 'Location': '512 Wilson Crescent'}])

pd.merge(staff_df, student_df, how='left', left_on='Name', right_on='Name')

#### 13 - Multi-Indexação

In [ ]:
staff_df = pd.DataFrame([{'First Name': 'Kelly', 'Last Name': 'Desjardins', 'Role': 'Director of HR'},
                         {'First Name': 'Sally', 'Last Name': 'Brooks', 'Role': 'Course liasion'},
                         {'First Name': 'James', 'Last Name': 'Wilde', 'Role': 'Grader'}])
student_df = pd.DataFrame([{'First Name': 'James', 'Last Name': 'Hammond', 'School': 'Business'},
                           {'First Name': 'Mike', 'Last Name': 'Smith', 'School': 'Law'},
                           {'First Name': 'Sally', 'Last Name': 'Brooks', 'School': 'Engineering'}])
staff_df
student_df
pd.merge(staff_df, student_df, how='inner', left_on=['First Name','Last Name'], right_on=['First Name','Last Name'])

# Python & Pandas Idiomáticos

#### 14 - Importar CSV Censo USA

In [ ]:
import pandas as pd
df = pd.read_csv('./CSV/census.csv')
df

#### 15 - Encadeamento de métodos (Idiomático)

In [ ]:
(df.where(df['SUMLEV']==50)
    .dropna()
    .set_index(['STNAME','CTYNAME'])
    .rename(columns ={'ESTIMATESBASE2010': 'Estimates Base 2010'}))

#### 16 - Forma Tradicional do código anterior

In [ ]:
df = df[df['SUMLEV']==50]
df.set_index(['STNAME','CTYNAME'], inplace=True)
df.rename(columns={'ESTIMATESBASE2010': 'Estimates Base 2010'})

#### 17 - Apply and Lambdas

In [ ]:
import numpy as np
rows = ['POPESTIMATE2010',
        'POPESTIMATE2011',
        'POPESTIMATE2012',
        'POPESTIMATE2013',
        'POPESTIMATE2014',
        'POPESTIMATE2015']
df.apply(lambda x: np.max(x[rows]), axis=1)

# Função Groupby()

#### 18 - Exclusão da totalização dos estados (==40)

In [ ]:
df =df[df['SUMLEV']==50]
df

#### 19 - Iterar todos os estados e gerar uma lista dos números com a média da população

In [ ]:
%%timeit -n 10
for state in df['STNAME'].unique():
    avg = np.average(df.where(df['STNAME']==state).dropna()['CENSUS2010POP'])
    print('Counties in state ' + state + ' have an average population of ' + str(avg))

#### 20 - Função Groupby()

In [ ]:
%%timeit -n 10
for group, frame in df.groupby('STNAME'):
    avg = np.average(frame['CENSUS2010POP'])
    print('Counties in state ' + group + ' have an average population of ' + str(avg))

#### 21 - Método agg() - construir um data frame sumário, com a população média por Estado

In [ ]:
df.groupby('STNAME').agg({'CENSUS2010POP': np.average})

#### 22 - Função Groupby (Série vs. Dataframe)

In [ ]:
print(type(df.groupby(level=0)['POPESTIMATE2010','POPESTIMATE2011']))
print(type(df.groupby(level=0)['POPESTIMATE2010']))

#### 23 - Na Série

In [ ]:
(df.set_index('STNAME').groupby(level=0)['CENSUS2010POP']
    .agg({'avg': np.average, 'sum': np.sum}))

#### 24 - No Dataframe

In [ ]:
(df.set_index('STNAME').groupby(level=0)['POPESTIMATE2010','POPESTIMATE2011']
    .agg({'avg': np.average, 'sum': np.sum}))

#### 25 - Comportamento não esperado dada a mudança de rotulagem

In [ ]:
(df.set_index('STNAME').groupby(level=0)['POPESTIMATE2010','POPESTIMATE2011']
    .agg({'POPESTIMATE2010': np.average, 'POPESTIMATE2011': np.sum}))

# Tabelas Pivot

#### 26 - Carregar conjunto de dados "car.csv"

In [ ]:
df = pd.read_csv('./CSV/cars.csv')

In [ ]:
df.head()

#### 27 - Comparação da marca de veículos elétricos contra os anos e fazer essa comparação em termos de capacidade da bateria

In [ ]:
df.pivot_table(values='(kW)', index='YEAR', columns='Make', aggfunc=np.mean)

#### 28 - Lista de funções diferentes para aplicar

In [ ]:
df.pivot_table(values='(kW)', index='YEAR', columns='Make', aggfunc=[np.mean,np.min], margins=True)

# Datas no Pandas

#### 29 - Timestamp

In [ ]:
pd.Timestamp('9/1/2017 10:05AM')

#### 30 - Period

In [ ]:
pd.Period('1/2017')

In [ ]:
pd.Period('3/1/2017')

#### 31 - DatetimeIndex

In [ ]:
t1 = pd.Series(list('abc'), [pd.Timestamp('2016-09-01'), pd.Timestamp('2016-09-02'), pd.Timestamp('2016-09-03')])
t1

#### 32 - Tipo da série

In [ ]:
type(t1.index)

#### 33 - PeriodIndex

In [ ]:
t2 = pd.Series(list('def'), [pd.Period('2016-09'), pd.Period('2016-10'), pd.Period('2016-11')])
t2

In [ ]:
type(t2.index)

#### 34 - Novo DataFrame para conversão  to_datetime()

In [ ]:
d1 = ['2 June 2013', 'Aug 29, 2014', '2015-06-26', '7/12/16']
ts3 = pd.DataFrame(np.random.randint(10, 100, (4,2)), index=d1, columns=list('ab'))
ts3

#### 35 - Usando to_datetime()

In [ ]:
ts3.index = pd.to_datetime(ts3.index)
ts3

#### 36 - Data no formato Europeu

In [ ]:
pd.to_datetime('4.7.12', dayfirst=True)

#### 37 - Timedeltas

In [ ]:
pd.Timestamp('9/3/2016')-pd.Timestamp('9/1/2016')

#### 38 - Encontrar datas e horas com Timedeltas

In [ ]:
pd.Timestamp('9/2/2016 8:10AM') + pd.Timedelta('12D 3H')

#### 39 - Método date_range()

In [ ]:
dates = pd.date_range('10-01-2016', periods=9, freq='2W-SUN')
dates

#### 40 - Introduzindo datas aleatórias

In [ ]:
df = pd.DataFrame({'Count 1': 100 + np.random.randint(-5, 10, 9).cumsum(),
                  'Count 2': 120 + np.random.randint(-5, 10, 9)}, index=dates)
df

#### 41 - Podemos verificar qual o dia da semana

In [ ]:
df.index.weekday_name

#### 42 - Diferença entre datas

In [ ]:
df.diff()

#### 43 - Função resample()

In [ ]:
df.resample('M').mean()

#### 44 - Indexação parcial em datas

In [ ]:
df['2017']

In [ ]:
df['2016-12']

In [ ]:
df['2016-12':]

#### 45 - Preenchendo valores ausentes

In [ ]:
df.asfreq('W', method='ffill')

#### 46 - Plotando

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

df.plot()